In [1]:
import networkx as nx
import csv
import numpy as np
import pandas as pd
import os
from collections import defaultdict
from node2vec import Node2Vec
from gensim.models import Word2Vec

import keras
import tensorflow as tf
from keras.layers import Dense, Dropout,Reshape, Activation
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers.merge import Concatenate

from sklearn.model_selection import KFold 

Using TensorFlow backend.


In [5]:
                                                # SETTINGS
nb_emb_w2v = 30
nb_topics = 24
w2vModel = Word2Vec.load("Files/word2vec_30.model")


In [6]:
                        # FORMAT TRAINING : SEPARATE GRAPH INFORMATION FROM TEXT INFORMATION

nlpDataframe2 = pd.read_csv("Files/nlpTrain.csv")

Link = nlpDataframe2.pop('Link')
CosSimilarityDf = nlpDataframe2[['Cos_similarity']]
nlpDataframe2.pop('Cos_similarity')

topic_list = []

for i in range(nb_emb_w2v):
    topic_name = "1_Emb "+ str(i)
    topic_list.append(topic_name)

for i in range(nb_emb_w2v):
    topic_name = "2_Emb "+ str(i)
    topic_list.append(topic_name)
    

embDf = pd.DataFrame(0.0, index=np.arange(nlpDataframe2.shape[0]), columns= topic_list)


for index,row in nlpDataframe2.iterrows():
    #print(index)
    if (index == 50000):
        print (50000)
        
    idx1 = int(row["Node1"])
    idx2 = int(row["Node2"])
    
    try :
        node1_values = w2vModel[str(idx1)]
    except KeyError:
        node1_values = [0.0 for i in range(nb_emb_w2v)]
    
    try :
        node2_values = w2vModel[str(idx2)]
    except KeyError:
        node2_values = [0.0 for i in range(nb_emb_w2v)]

    for i,(v1,v2) in enumerate(zip(node1_values, node2_values)):
        
        n1_emb_name = "1_Emb "+ str(i)
        n2_emb_name = "2_Emb "+ str(i)
        
        embDf.at[index, n1_emb_name] = v1
        embDf.at[index, n2_emb_name] = v2

pop1 = nlpDataframe2.pop("Node1")
pop2 = nlpDataframe2.pop("Node2")

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


50000


In [7]:
embDf.head()

,1_Emb 0,1_Emb 1,1_Emb 2,1_Emb 3,1_Emb 4,1_Emb 5,1_Emb 6,1_Emb 7,1_Emb 8,1_Emb 9,...,2_Emb 20,2_Emb 21,2_Emb 22,2_Emb 23,2_Emb 24,2_Emb 25,2_Emb 26,2_Emb 27,2_Emb 28,2_Emb 29
0,-1.149851,4.094054,1.980226,1.061261,-1.293822,4.226110,3.173213,1.145736,-0.329324,2.007836,...,-0.039129,2.486328,0.716066,5.309447,-1.214457,1.535208,2.385972,-0.819087,0.302707,0.007153
1,0.713437,-0.186107,-0.167109,-0.464693,-0.660424,0.088300,0.691821,-0.293232,-0.505323,0.071237,...,0.037106,-0.137260,0.734854,0.102968,-0.583279,-0.253284,0.052709,0.452585,0.320666,0.045048
2,-2.036839,3.259414,-0.206073,0.603257,-0.867595,0.086645,-0.859987,2.220953,2.649518,4.457850,...,-0.999971,2.052428,1.316718,2.722214,-0.695560,-0.528354,1.626114,0.471445,-0.419526,1.729401
3,-4.191712,4.765485,-2.723879,1.253531,-3.725234,6.303171,-1.767110,2.476411,-4.272335,7.383749,...,-0.478634,0.101077,0.530740,0.227917,0.236640,-0.128904,-0.102085,-0.254686,0.242145,0.343943
4,-0.475622,-2.404068,-1.396102,-0.700633,0.667871,2.030433,-1.196424,-0.880149,-1.205218,-4.978656,...,-0.980820,0.019812,-1.226661,0.087983,0.270624,-0.388618,0.103238,-1.320837,-0.078508,-0.743266


In [10]:
                                    # K FOLD MODEL TRAINING FOR NEURAL NET
listModel = []

kf = KFold(n_splits=10, random_state=42, shuffle=True)# Define the split - into 10 folds 

for train_index, test_index in kf.split(embDf):
    
    X_train, X_test = embDf.loc[train_index], embDf.loc[test_index]
    y_train, y_test = Link[train_index], Link[test_index]
    model = Sequential()
    model.add(Dense(60, activation='relu', input_dim = nb_emb_w2v*2))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=6, batch_size=32)
    listModel.append(model)

Epoch 1/6
408417/408417 [==============================] - 14s 35us/step - loss: 0.1422 - accuracy: 0.9485
Epoch 2/6
408417/408417 [==============================] - 13s 33us/step - loss: 0.1074 - accuracy: 0.9631
Epoch 3/6
408417/408417 [==============================] - 13s 32us/step - loss: 0.1012 - accuracy: 0.9660
Epoch 4/6
408417/408417 [==============================] - 12s 30us/step - loss: 0.1019 - accuracy: 0.9671
Epoch 5/6
408417/408417 [==============================] - 13s 32us/step - loss: 0.1051 - accuracy: 0.9684
Epoch 6/6
408417/408417 [==============================] - 14s 33us/step - loss: 0.1003 - accuracy: 0.9685
Epoch 1/6
408417/408417 [==============================] - 16s 39us/step - loss: 0.1429 - accuracy: 0.9479
Epoch 2/6
408417/408417 [==============================] - 13s 32us/step - loss: 0.1096 - accuracy: 0.9620
Epoch 3/6
408417/408417 [==============================] - 13s 33us/step - loss: 0.1027 - accuracy: 0.9651
Epoch 4/6
408417/408417 [============

In [11]:
# choose the index of the best performing net
index = 5
model = listModel[index]

In [12]:
                                           # FORMAT SUBMISSION DATASET

    
nlpTestDataframe = pd.read_csv('Files/nlpTest.csv')
CosSimilarityTestDf = nlpTestDataframe[['Cos_similarity']]
nlpTestDataframe.pop('Cos_similarity')

topic_list = []

for i in range(nb_emb_w2v):
    topic_name = "1_Emb "+ str(i)
    topic_list.append(topic_name)

for i in range(nb_emb_w2v):
    topic_name = "2_Emb "+ str(i)
    topic_list.append(topic_name)
    

embDfTest = pd.DataFrame(0.0, index=np.arange(nlpTestDataframe.shape[0]), columns= topic_list)

for index,row in nlpTestDataframe.iterrows():
    #print(index)
    if (index == 50000):
        print (50000)
        
    idx1 = int(row["Node1"])
    idx2 = int(row["Node2"])
    
    try :
        node1_values = w2vModel[str(idx1)]
    except KeyError:
        node1_values = [0.0 for i in range(nb_emb_w2v)]
    
    try :
        node2_values = w2vModel[str(idx2)]
    except KeyError:
        node2_values = [0.0 for i in range(nb_emb_w2v)]

    for i,(v1,v2) in enumerate(zip(node1_values, node2_values)):
        
        n1_emb_name = "1_Emb "+ str(i)
        n2_emb_name = "2_Emb "+ str(i)
        
        embDfTest.at[index, n1_emb_name] = v1
        embDfTest.at[index, n2_emb_name] = v2
        
pop1 = nlpTestDataframe.pop('Node1')
pop2 = nlpTestDataframe.pop('Node2')

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


50000


In [13]:
predictions = model.predict_classes(embDfTest)
submissionDf = pd.DataFrame()
submissionDf["predicted"] = predictions.transpose()[0]
submissionDf.index.name = 'id'
file_name = "Results/predictionsNodeNeural.csv"
submissionDf.to_csv(file_name, encoding='utf-8', index=True)

In [42]:
                                 # K FOLD TRAINING FOR GRADIENT BOOSTING
    
from sklearn import ensemble
    
n_estim = 100
n_split = 5
depth = 5
    
listModelBoosting = []

kf = KFold(n_splits=n_split, random_state=42, shuffle=True)# Define the split - into 10 folds 

val_scores = np.zeros((n_estim,), dtype=np.float64)

def heldout_score(clf, X_test, y_test):
    """compute deviance scores on ``X_test`` and ``y_test``. """
    score = np.zeros((n_estim,), dtype=np.float64)
    for i, y_pred in enumerate(clf.staged_decision_function(X_test)):
        score[i] = clf.loss_(y_test, y_pred)
    print(score)
    return score

for train_index, test_index in kf.split(embDf):
    
    X_train, X_test = embDf.loc[train_index], embDf.loc[test_index]
    y_train, y_test = Link[train_index], Link[test_index]

    clf = ensemble.GradientBoostingClassifier(learning_rate=0.1, n_estimators=n_estim, subsample=1.0,max_depth=depth)
    
    print('HERE')

    clf.fit(X_train, y_train)
    
    val_scores += heldout_score(clf, X_test, y_test)
    
    acc = clf.score(X_test, y_test)
    print("Accuracy: {:.4f}".format(acc))
    
    listModelBoosting.append(clf)

val_scores /= n_split

HERE
[1.2237279  1.14077849 1.07248285 1.01345167 0.96128956 0.91473894
 0.87439923 0.8389899  0.80525883 0.77700532 0.75236057 0.72767794
 0.70433068 0.6854349  0.66674258 0.64883572 0.63276792 0.61853466
 0.60493208 0.59222763 0.57997105 0.56835337 0.55833326 0.54901494
 0.53955862 0.53105689 0.5224053  0.51391077 0.50718957 0.4993488
 0.49272122 0.48678077 0.47914542 0.47213859 0.46565521 0.46212669
 0.45665881 0.45148553 0.44562116 0.44062262 0.43648843 0.4312721
 0.42642521 0.42274965 0.41893074 0.41398815 0.41085221 0.40729297
 0.40266384 0.39871735 0.39476062 0.39158684 0.38779425 0.38409731
 0.38055927 0.37708259 0.37447749 0.37119661 0.36717868 0.36409484
 0.36267795 0.359175   0.35798534 0.35526529 0.35149437 0.34867597
 0.34671459 0.34452724 0.34224557 0.33902041 0.33683739 0.3342327
 0.33161667 0.3291595  0.32671585 0.32454357 0.32234907 0.32060172
 0.31935673 0.31670757 0.31468416 0.31293255 0.31097214 0.30927235
 0.30710533 0.30465204 0.30244402 0.30077677 0.29928244 0.29

In [43]:
clf = listModelBoosting[1]

In [44]:
predictionsBoosting = clf.predict(embDfTest.loc[:])
submissionDfBoosting = pd.DataFrame()
submissionDfBoosting["predicted"] = predictionsBoosting.transpose()
submissionDfBoosting.index.name = 'id'
file_name_boosting = "Results/predictionsNodeBoosting.csv"
submissionDfBoosting.to_csv(file_name_boosting, encoding='utf-8', index=True)

In [45]:
submissionDfBoosting.head()

,predicted
id,
0,1
1,1
2,0
3,0
4,1


In [63]:
                                 # K FOLD TRAINING FOR LOGISTIC REGRESSION
    
    
from sklearn.linear_model import LogisticRegression

n_split = 5
C_value = 1.0
loss = 'l2'
solv = 'lbfgs'

listModelLogistic = []

kf = KFold(n_splits=n_split, random_state=42, shuffle=True)# Define the split - into 10 folds 


for train_index, test_index in kf.split(embDf):
    
    X_train, X_test = embDf.loc[train_index], embDf.loc[test_index]
    y_train, y_test = Link[train_index], Link[test_index]

    clf = LogisticRegression(penalty = loss, C = C_value, solver = solv)
    
    print('HERE')

    clf.fit(X_train, y_train)
    
    acc = clf.score(X_test, y_test)
    
    print("Accuracy: {:.4f}".format(acc))
    
    listModelLogistic.append(clf)

HERE
Accuracy: 0.8317
HERE
Accuracy: 0.8335
HERE
Accuracy: 0.8321
HERE
Accuracy: 0.8330
HERE
Accuracy: 0.8338


In [64]:
clf = listModelLogistic[4]

In [65]:
predictionsLogistic = clf.predict(embDfTest.loc[:])
submissionDfLogistic = pd.DataFrame()
submissionDfLogistic["predicted"] = predictionsLogistic.transpose()
submissionDfLogistic.index.name = 'id'
file_name_logistic = "Results/predictionsNodeLogistic.csv"
submissionDfLogistic.to_csv(file_name_logistic, encoding='utf-8', index=True)